In [2]:
from src.data.consts import FILE_PATH
import pandas as pd
import zipfile



df = pd.read_csv('../data/interm/preprocessed_filtered.csv')

In [3]:
tox_df = df[['reference', 'ref_tox']].copy()
tox_df = tox_df[:40000]

In [4]:
tox_df.head()

,reference,ref_tox
0,'m gon na child ... ... genet disord 's gon na...,0.950956
1,main short black peopl back .,0.963680
2,"spirit curs , walk back road , waterway , find...",0.842509
3,"night , li ' l dice satisfi thirst kill , thou...",0.598596
4,"real life start first time fuck , kid .",0.998222


In [9]:
from sklearn.model_selection import train_test_split

X = tox_df['reference']  # тексты
y = tox_df['ref_tox']    # уровни токсичности

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.125, random_state=42)


In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)
X_train_tfidf = X_train_tfidf.astype('float32')



In [11]:
import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train_tfidf.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])


In [12]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [13]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train_tfidf.toarray(), y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test_tfidf.toarray(), y_test))


In [14]:
batch_size = 32
epochs = 20

model.fit(train_dataset.batch(batch_size), epochs=epochs, validation_data=test_dataset.batch(batch_size))

Epoch 1/20
1094/1094 [==============================] - 12s 10ms/step - loss: 0.2219 - accuracy: 0.0000e+00 - val_loss: 0.2100 - val_accuracy: 0.0000e+00
Epoch 2/20
1094/1094 [==============================] - 10s 10ms/step - loss: 0.2037 - accuracy: 0.0000e+00 - val_loss: 0.2122 - val_accuracy: 0.0000e+00
Epoch 3/20
1094/1094 [==============================] - 10s 9ms/step - loss: 0.1981 - accuracy: 0.0000e+00 - val_loss: 0.2159 - val_accuracy: 0.0000e+00
Epoch 4/20
1094/1094 [==============================] - 10s 9ms/step - loss: 0.1905 - accuracy: 0.0000e+00 - val_loss: 0.2207 - val_accuracy: 0.0000e+00
Epoch 5/20
1094/1094 [==============================] - 13s 12ms/step - loss: 0.1826 - accuracy: 0.0000e+00 - val_loss: 0.2260 - val_accuracy: 0.0000e+00
Epoch 6/20
1094/1094 [==============================] - 14s 13ms/step - loss: 0.1777 - accuracy: 0.0000e+00 - val_loss: 0.2310 - val_accuracy: 0.0000e+00
Epoch 7/20
1094/1094 [==============================] - 14s 13ms/step - loss: 

In [15]:
y_pred = model.predict(X_test_tfidf.toarray())

157/157 [==============================] - 1s 3ms/step


In [16]:
from sklearn.metrics import accuracy_score

# Преобразуйте истинные метки тестовых данных в бинарный формат, если они не таковы
y_test_binary = (y_test > 0.5).astype(int)
y_pred_binary = (y_pred > 0.5).astype(int)


# Оцените точность
accuracy = accuracy_score(y_test_binary, y_pred_binary)
print(f'Accuracy: {accuracy:.2f}')


Accuracy: 1.00


In [17]:
# Экспортируйте модель в файл
model.save('model.h5')

C:\Users\lesak\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
